In [2]:
import os
import pandas as pd
import numpy as np
import random
from math import ceil
from time import time
import torch
from lightgbm import LGBMClassifier, Dataset

In [3]:
import matplotlib.pyplot as plt

In [4]:
%matplotlib notebook

In [7]:
dataDir= '../data'
max_hist_len = 12

#### 填充缺失值

In [29]:
file_ls = [
#     ['invite_info_[3840, 3860]-[3861, 3867]_quest_hist_meanpool_dist',
#             'invite_info_test-[3847, 3867]-[3868, 3874]_quest_hist_meanpool_dist'
#             ],
#            ['invite_info_[3840, 3860]-[3861, 3867]_quest_hist_minpool_dist',
#             'invite_info_test-[3847, 3867]-[3868, 3874]_quest_hist_minpool_dist',
#            ],
#            [ 'invite_info_[3840, 3860]-[3861, 3867]_quest_hist_maxpool_dist',
#             'invite_info_test-[3847, 3867]-[3868, 3874]_quest_hist_maxpool_dist',
#            ],
#            ['invite_info_[3840, 3860]-[3861, 3867]_quest_hist_maxpool_dist_describe',
#            'invite_info_test-[3847, 3867]-[3868, 3874]_quest_hist_maxpool_dist_describe',
#            ],
#            [
#                'invite_info_[3840, 3860]-[3861, 3867]_quest_hist_meanpool_dist_describe',
#             'invite_info_test-[3847, 3867]-[3868, 3874]_quest_hist_meanpool_dist_describe',
#            ],
#            [
#                 'invite_info_[3840, 3860]-[3861, 3867]_quest_hist_minpool_dist_describe',
#                'invite_info_test-[3847, 3867]-[3868, 3874]_quest_hist_minpool_dist_describe',
#            ] 
            [
                'invite_info_[3840, 3860]-[3861, 3867]_quest-hist_topics_quest-user_follow_topics_quest-inn_topics_dist',
                'invite_info_test-[3847, 3867]-[3868, 3874]_quest-hist_topics_quest-user_follow_topics_quest-inn_topics_dist',
            ]
          ]

In [30]:
#for file in file_ls:
#    df = pd.read_csv(os.path.join(dataDir,'%s.csv' %(file, )), sep= '\t', index_col = 0, na_values= '-1',
#                    nrows= 500
#                    )
#    print(df.head().index)


#for file in file_ls:
#    df = pd.read_csv(os.path.join(dataDir,'%s.csv' %(file, )), sep= '\t', index_col = 0, na_values= '-1',
#                     nrows= 500
#                    )
#    print(df.columns)

for train_fileStr, test_fileStr in file_ls:
    df = pd.read_csv(os.path.join(dataDir, '%s.csv' %(train_fileStr, )), sep= '\t', index_col= 0, na_values = '-1')

    print(df.isna().sum() / len(df))
    df_mean = df.mean()
    df_filled= df.fillna(df_mean)

    df_filled.to_csv(os.path.join(dataDir, '%s_na_filled_with_mean.csv' %(train_fileStr,)), sep= '\t')

    test_df = pd.read_csv(os.path.join(dataDir, '%s.csv' %(test_fileStr, )), sep= '\t', index_col= 0, na_values = '-1')
    test_df_filled= test_df.fillna(df_mean)

    test_df_filled.to_csv(os.path.join(dataDir, '%s_na_filled_with_mean.csv' %(test_fileStr,)), sep= '\t')

quest_user_follow_topics_euclid_dist_min     0.002076
quest_user_follow_topics_euclid_dist_max     0.002076
quest_user_follow_topics_euclid_dist_mean    0.002076
quest_user_follow_topics_euclid_dist_std     0.002076
quest_topics_inn_euclid_dist_max             0.002076
quest_topics_inn_euclid_dist_mean            0.002076
quest_topics_inn_euclid_dist_std             0.002076
quest_hist_topics_euclid_dist_min            0.417815
quest_hist_topics_euclid_dist_max            0.417815
quest_hist_topics_euclid_dist_std            0.417815
quest_hist_topics_euclid_dist_mean           0.417815
dtype: float64


In [25]:
train_fileStr = 'invite_info_[3840, 3860]-[3861, 3867]_quest_user_interest_topics_cosine_dist_describe'
test_fileStr= 'invite_info_test-[3847, 3867]-[3868, 3874]_quest_user_interest_topics_cosine_dist_describe'

df = pd.read_csv(os.path.join(dataDir, '%s.csv' %(train_fileStr, )), sep= '\t', index_col= 0, na_values = '-1')

print(df.isna().sum() / len(df))
df_mean = df.mean()
df_filled= df.fillna(df_mean)

df_filled.to_csv(os.path.join(dataDir, '%s_na_filled_with_mean.csv' %(train_fileStr,)), sep= '\t')

test_df = pd.read_csv(os.path.join(dataDir, '%s.csv' %(test_fileStr, )), sep= '\t', index_col= 0, na_values = '-1')
test_df_filled= test_df.fillna(df_mean)

test_df_filled.to_csv(os.path.join(dataDir, '%s_na_filled_with_mean.csv' %(test_fileStr,)), sep= '\t')

quest_user_interest_topics_cosine_weight_min_dist              0.157584
quest_user_interest_topics_cosine_weight_max_dist              0.157584
quest_user_interest_topics_cosine_weight_mean_dist             0.157584
quest_user_interest_topics_cosine_min_dist_top3_weight_sum     0.157536
quest_user_interest_topics_cosine_min_dist_tail3_weight_sum    0.157536
quest_user_interest_topics_cosine_min_dist_top_weight          0.157536
quest_user_interest_topics_cosine_min_dist_tail_weight         0.157536
dtype: float64


In [24]:
train_fileStr = 'invite_info_[3840, 3860]-[3861, 3867]_quest_user_follow_topics_cosine_dist_describe'
test_fileStr= 'invite_info_test-[3847, 3867]-[3868, 3874]_quest_user_follow_topics_cosine_dist_describe'

df = pd.read_csv(os.path.join(dataDir, '%s.csv' %(train_fileStr, )), sep= '\t', index_col= 0, na_values = '-1')

print(df.isna().sum() / len(df))
df_mean = df.mean()
df_filled= df.fillna(df_mean)

df_filled.to_csv(os.path.join(dataDir, '%s_na_filled_with_mean.csv' %(train_fileStr,)), sep= '\t')

test_df = pd.read_csv(os.path.join(dataDir, '%s.csv' %(test_fileStr, )), sep= '\t', index_col= 0, na_values = '-1')
test_df_filled= test_df.fillna(df_mean)

test_df_filled.to_csv(os.path.join(dataDir, '%s_na_filled_with_mean.csv' %(test_fileStr,)), sep= '\t')

quest_user_follow_topics_cosine_dist_min          0.133236
quest_user_follow_topics_cosine_dist_max          0.133236
quest_user_follow_topics_cosine_dist_mean         0.133236
quest_user_follow_topics_cosine_dist_std          0.133236
quest_user_follow_topics_cosine_dist_top3_mean    0.133236
dtype: float64


In [23]:
train_fileStr = 'invite_info_[3840, 3860]-[3861, 3867]_quest_topics_inn_euclid_dist_describe'
test_fileStr= 'invite_info_test-[3847, 3867]-[3868, 3874]_quest_topics_inn_euclid_dist_describe'

df = pd.read_csv(os.path.join(dataDir, '%s.csv' %(train_fileStr, )), sep= '\t', index_col= 0, na_values = '-1')

print(df.isna().sum() / len(df))
df_mean = df.mean()
df_filled= df.fillna(df_mean)

df_filled.to_csv(os.path.join(dataDir, '%s_na_filled_with_mean.csv' %(train_fileStr,)), sep= '\t')

test_df = pd.read_csv(os.path.join(dataDir, '%s.csv' %(test_fileStr, )), sep= '\t', index_col= 0, na_values = '-1')
test_df_filled= test_df.fillna(df_mean)

test_df_filled.to_csv(os.path.join(dataDir, '%s_na_filled_with_mean.csv' %(test_fileStr,)), sep= '\t')

quest_topics_inn_euclid_dist_max     0.002076
quest_topics_inn_euclid_dist_mean    0.002076
quest_topics_inn_euclid_dist_std     0.002076
dtype: float64


In [ ]:
train_fileStr = 'invite_info_test-[3847, 3867]-[3868, 3874]_quest_user_follow_topics_euclid_dist_describe'
test_fileStr= 'invite_info_test-[3847, 3867]-[3868, 3874]_quest_topics_inn_euclid_dist_describe'

df = pd.read_csv(os.path.join(dataDir, '%s.csv' %(train_fileStr, )), sep= '\t', index_col= 0, na_values = '-1')

print(df.isna().sum() / len(df))
df_mean = df.mean()
df_filled= df.fillna(df_mean)

df_filled.to_csv(os.path.join(dataDir, '%s_na_filled_with_mean.csv' %(train_fileStr,)), sep= '\t')

test_df = pd.read_csv(os.path.join(dataDir, '%s.csv' %(test_fileStr, )), sep= '\t', index_col= 0, na_values = '-1')
test_df_filled= test_df.fillna(df_mean)

test_df_filled.to_csv(os.path.join(dataDir, '%s_na_filled_with_mean.csv' %(test_fileStr,)), sep= '\t')

quest_topics_inn_euclid_dist_max     0.002076
quest_topics_inn_euclid_dist_mean    0.002076
quest_topics_inn_euclid_dist_std     0.002076
dtype: float64
